In [1]:
from pysat import solvers
from itertools import combinations

Biến logic cho ô [x, y] của matrix(m, n) có giá trị tuyệt đối là x * n + y. Nếu dương là Trap, ngược là lại Gem

Dễ dàng chứng minh được rằng: <br >
Ít nhất m literals trong n literals là đúng khi và chỉ khi với mỗi bộ hoán vị có m - n + 1 literals, luôn có ít nhất 1 literal đúng

In [2]:
def read_matrix(file_name):
    with open(file_name) as f:
        matrix = []
        for line in f:
            matrix.append([int(x) if x != '_' else x for x in line.strip().split()])
    return matrix

# Generate a clause that is true if and only if exactly n literals of the literals are true
def generate_cnf_exactly_n(n, literals):
    cnf = []
    size = len(literals)

    #At least n of the literals are true
    for comb in combinations(literals, size - n + 1):
        cnf.append(list(comb))

    #At most n of the literals are true 
    #Equivalent to at least size-n+1 of the literals are false
    for comb in combinations(literals, n+1):
        cnf.append([-lit for lit in comb])

    return cnf

# Get no. cell of Trap and Gem around a cell
def get_around(matrix, i, j):
    m = len(matrix)
    n = len(matrix[0])
    move = [[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1], [1, 0], [1, 1]]
    around = []
    for [x, y] in move:
        if 0 <= i + x < m and 0 <= j + y < n and matrix[i + x][j + y] == '_':
            around.append((i + x) * n + (j + y))

    return around


In [3]:
def generate_cnf(matrix):
    m = len(matrix)
    n = len(matrix[0])
    cnf = []

    for i in range(m):
        for j in range(n):
            if matrix[i][j] != '_':
                around = get_around(matrix, i, j)
                cnf_cell = generate_cnf_exactly_n(matrix[i][j], around)
                
                for clause in cnf_cell:
                    if clause not in cnf:
                        cnf.append(clause)

    return cnf

In [4]:
def solve_cnf_pysat(cnf):
    solver = solvers.Glucose3()
    for clause in cnf:
        solver.add_clause(clause)

    if solver.solve():
        model = solver.get_model()
        return model
    
    return None

In [5]:
def solve_cnf_my_solution(cnf):
    #TODO: Implement own cnfs solver
    return None

In [6]:
def solve_cnf_brute_force(cnf):
    #TODO: Implement a brute force cnf solver
    return None

In [7]:
def solve_cnf_back_tracking(cnf):
    #TODO: Implement a backtracking cnf solver
    return None

In [8]:
import copy
def write_solution(model, matrix, output_file):
    matrix_copy = []
    m = len(matrix[0])
    if model is not None:
        matrix_copy = copy.deepcopy(matrix)
        for x in model:
            y = x if x > 0 else -x
            if matrix_copy[y // m][y % m] == '_':
                matrix_copy[y // m][y % m] = 'T' if x > 0 else 'G'
    
    with open(output_file, 'w') as f:
        for row in matrix_copy:
            f.write(' '.join(str(x) for x in row) + '\n')

In [9]:
import os
import time

folder_path_input = 'testcases/input'

folder_path_output = ['output-pysat', 'output-my-solution', 'output-brute-force', 'output-back-tracking']
functions_solve_cnf = [solve_cnf_pysat, solve_cnf_my_solution, solve_cnf_brute_force, solve_cnf_back_tracking]

num_functions = len(functions_solve_cnf)

for index, file_name in enumerate(os.listdir(folder_path_input)):
    print(f"Test {index}: {file_name}")
    for i in range(num_functions):
        matrix = read_matrix(os.path.join(folder_path_input, file_name))
        cnf = generate_cnf(matrix)

        start_time = time.time()    
        model = functions_solve_cnf[i](cnf)
        end_time = time.time()
        print(f"{functions_solve_cnf[i].__name__}: {end_time - start_time:.8f}s")

        write_solution(model, matrix,
                        os.path.join("testcases", folder_path_output[i], f"output{index}.txt"))
    print()

Test 0: input0.txt
solve_cnf_pysat: 0.00056028s
solve_cnf_my_solution: 0.00000143s
solve_cnf_brute_force: 0.00000143s
solve_cnf_back_tracking: 0.00000095s

Test 1: input1.txt
solve_cnf_pysat: 0.00021839s
solve_cnf_my_solution: 0.00000215s
solve_cnf_brute_force: 0.00000143s
solve_cnf_back_tracking: 0.00000143s

